# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686510


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:01<00:25,  1.09it/s]

Rendering models:  10%|█         | 3/30 [00:04<00:41,  1.53s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:30,  1.16s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:40,  1.60s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:14<00:01,  2.78it/s]

Rendering models:  90%|█████████ | 27/30 [00:14<00:01,  2.63it/s]

Rendering models:  93%|█████████▎| 28/30 [00:15<00:00,  2.82it/s]

Rendering models:  97%|█████████▋| 29/30 [00:15<00:00,  2.80it/s]

not-logged-in-61893b5c98750b56434b     0.001531
not-logged-in-20a275b53abec8db9f41     0.005649
shirogane                              0.000284
Dynamatt                               0.000476
yooman12                               0.000305
kjkavene                               0.000442
nsyee                                  0.000424
jmfranci                               0.000686
tosnyder                               0.000583
ncaro                                  0.001609
not-logged-in-66880d921e998997b392     0.000530
not-logged-in-9a45911eb71184fe7c7f     0.006359
not-logged-in-5f30b2bb9b7f9d49da41     0.002893
not-logged-in-09330e0b15263888ee9d     0.000879
not-logged-in-038a8033098ea41002e3     0.040288
Hollywood1116                          0.001606
jmartinez468                           0.004259
pangeli5                               0.000514
acapirala                              0.000352
not-logged-in-04ed9ae880f18762cd9e    10.721683
Lavadude                               0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())